# MODELO RAG DE HUGGINGFACE Y LANGCHAIN



*   BENEFICIOS: LA BÚSQUEDA SEMÁNTICA DE FAISS FUNCIONA BIEN
*   CONTRAS: LA RESPUESTA DEL SISTEMA DE QUESTION ANWERING ES DEMASIADO ESCUETA



In [ ]:
!pip install langchain -q
!pip install faiss-cpu -q
#!pip install chromadb
!pip install sentence_transformers -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.2 MB/s eta 0:00:00


In [ ]:
from langchain.document_loaders import TextLoader

In [ ]:
# base de conocimiento en formato txt
!wget --no-check-certificate 'link de descarga' -O extraccion.txt

--2023-09-21 14:22:11--  https://drive.google.com/uc?export=download&id=1vOOnJi09Z2eNkDkSTUYKjmidHRApjWA0
Resolving drive.google.com (drive.google.com)... 74.125.143.100, 74.125.143.113, 74.125.143.101, ...
Connecting to drive.google.com (drive.google.com)|74.125.143.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0g-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/uocq1qoocrqactudmjggo3nm9laob9gn/1695306075000/18173374041057561268/*/1vOOnJi09Z2eNkDkSTUYKjmidHRApjWA0?e=download&uuid=d69f2093-5349-405a-927f-ce7259a2c82a [following]
--2023-09-21 14:22:12--  https://doc-0g-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/uocq1qoocrqactudmjggo3nm9laob9gn/1695306075000/18173374041057561268/*/1vOOnJi09Z2eNkDkSTUYKjmidHRApjWA0?e=download&uuid=d69f2093-5349-405a-927f-ce7259a2c82a
Resolving doc-0g-4c-docs.googleusercontent.com (doc-0g-4c-docs.googleusercontent.com)... 108.177.126.132, 2a00

In [ ]:
#Para archivos txt

loader = TextLoader('extraccion.txt')
document = loader.load()

## 2 - Crear chunks

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
with open('extraccion.txt') as f:
    extraccion_texto = f.read()



In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=600,
    chunk_overlap=0,
    length_function = len,
    is_separator_regex=False,
    )

In [ ]:
chunks = text_splitter.create_documents([extraccion_texto])

In [ ]:
len(chunks)

640

In [ ]:
chunks[13]


Document(page_content='campos con su nombre, tipo y valor, cuya semántica (significado) está bien definida y que suelen estar \nnormalizados. \nEn  ,  Merrill  Lynch  afirmó  que  "los  datos  no  estructurados  comprenden  la  gran \nmayoría  de  los  datos  que  se  encuentran  en  una  organización,  llegando  en  algunas \nestimaciones al % de todos los datos de la organización” [Christopher C. Shilakes, \nJulie Tylman (). Enterprise Information Portals. Merrill Lynch]. \nSin embargo, en general se asume que la información estructurada tiene un valor muy superior a la no estructurada', metadata={})

### 3 - Crear Embeddings

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings


In [ ]:
# DOS MODELOS:

'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2' # 471Mb
'sentence-transformers/paraphrase-multilingual-mpnet-base-v2' #1.11Gb

'sentence-transformers/paraphrase-multilingual-mpnet-base-v2'

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2")

Crear embeddings de todo el texto

In [ ]:
# Chroma:
#from langchain.vectorstores import Chroma
#FAISS:
from langchain.vectorstores import FAISS


In [ ]:
#con CHROMA (funciona mejor FAISS, más abajo)


from langchain.vectorstores import Chroma

db = Chroma.from_documents(chunks, embeddings)

ImportError: ignored

In [ ]:
#con FAISS (ESTE SÍ)

from langchain.vectorstores import FAISS
db = FAISS.from_documents(chunks, embeddings)

NameError: ignored

In [ ]:
pregunta = "¿Qué es un chunk?"
docs = db.similarity_search(pregunta)
print(docs)

NameError: ignored

In [ ]:
#CONVERTIMOS TEXTO A CADENA

class Document:
    def __init__(self, text):
        self.text = text


document_list = [docs]

mi_string=' '.join([str(element) for element in document_list])

print(mi_string)

[Document(page_content='Otro modelo muy conocido es el de Robert Plutchik, que propuso un enfoque de clasificación representado en la \nfamosa "rueda  de  las  emociones"  para explicar su propuesta de forma gráfica. Su esquema consiste en ocho \nemociones bipolares básicas: alegría-tristeza (joy-sadness), confianza-asco (trust-disgust), ira-miedo (anger-fear) y \nsorpresa-anticipación (surprise-anticipation). \n \n \n \n \n \n \n \n \n \n \n \n \n \n Clasificación automática \nRueda de las emociones de Plutchik \nIgual que en el análisis de sentimientos, es esencial considerar la negación para poder identificar correctamente la', metadata={}), Document(page_content='como  a  nivel  de  cada  entidad,  como  se  muestra  en  la  figura  siguiente.  El  modelo  empleado  es  el  modelo  de \nemociones  de  Paul  Ekman,  con  las  emociones  básicas:  alegría  (joy),  tristeza  (sadness),  miedo  (fear),  asco \n(disgust) e ira (anger), evaluadas de  a %. \nAnálisis de emoción \nPrueba c

In [ ]:
#LIMPIAMOS CADENA DE TEXTO

import re
# Utiliza una expresión regular para eliminar caracteres no deseados, pero conserva las letras con tilde
texto_limpio = re.sub(r'[^\x00-\x7FáéíóúÁÉÍÓÚüÜñÑ]+', ' ', mi_string)

# Otras correcciones
texto_limpio = re.sub(r'\s+', ' ', texto_limpio)

texto_limpio = re.sub(r'\s*\\n\s*', ' ', texto_limpio)

texto_limpio = re.sub(r'metadata={}', ' ', texto_limpio)

texto_limpio = re.sub(r'Document', ' ', texto_limpio)
texto_limpio = re.sub(r'\(page_content=', ' ', texto_limpio)
texto_limpio = re.sub(r'\),', ' ', texto_limpio)
texto_limpio = re.sub(r'\[\'aplanado \(flatten etc.', ' ', texto_limpio)
print(texto_limpio)


[  'Otro modelo muy conocido es el de Robert Plutchik, que propuso un enfoque de clasificación representado en la famosa "rueda de las emociones" para explicar su propuesta de forma gráfica. Su esquema consiste en ocho emociones bipolares básicas: alegría-tristeza (joy-sadness  confianza-asco (trust-disgust  ira-miedo (anger-fear) y sorpresa-anticipación (surprise-anticipation).              Clasificación automática Rueda de las emociones de Plutchik Igual que en el análisis de sentimientos, es esencial considerar la negación para poder identificar correctamente la',      'como a nivel de cada entidad, como se muestra en la figura siguiente. El modelo empleado es el modelo de emociones de Paul Ekman, con las emociones básicas: alegría (joy  tristeza (sadness  miedo (fear  asco (disgust) e ira (anger  evaluadas de a %. Análisis de emoción Prueba con otros textos y analiza los resultados obtenidos. . Google Cloud Natural Language De nuevo, accede al sitio web de la plataforma y desplaza 

In [ ]:
import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

In [ ]:
nuevo_texto = wrap_text_preserve_newlines(texto_limpio)

In [ ]:
print(nuevo_texto)

[  'Otro modelo muy conocido es el de Robert Plutchik, que propuso un enfoque de clasificación representado en
la famosa "rueda de las emociones" para explicar su propuesta de forma gráfica. Su esquema consiste en ocho
emociones bipolares básicas: alegría-tristeza (joy-sadness  confianza-asco (trust-disgust  ira-miedo (anger-
fear) y sorpresa-anticipación (surprise-anticipation).              Clasificación automática Rueda de las
emociones de Plutchik Igual que en el análisis de sentimientos, es esencial considerar la negación para poder
identificar correctamente la',      'como a nivel de cada entidad, como se muestra en la figura siguiente. El
modelo empleado es el modelo de emociones de Paul Ekman, con las emociones básicas: alegría (joy  tristeza
(sadness  miedo (fear  asco (disgust) e ira (anger  evaluadas de a %. Análisis de emoción Prueba con otros
textos y analiza los resultados obtenidos. . Google Cloud Natural Language De nuevo, accede al sitio web de la
plataforma y desplaza

In [ ]:
from transformers import pipeline

In [ ]:
nlp = pipeline("question-answering", model="IProject-10/xlm-roberta-base-finetuned-squad2")



In [ ]:
nlp(question=pregunta, context=nuevo_texto)

{'score': 0.18494448065757751,
 'start': 160,
 'end': 219,
 'answer': 'trozo ) a la secuencia de tokens que conforman una\nentidad.'}

# LANGCHAIN Y API DE CHATGPT



*   BENEFICIOS: ¡FUNCIONA A LA PERFECCIÓN!
*   CONTRAS: es de pago :(
  





In [ ]:
!pip install openai

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = 'YOUR API KEY'

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain

In [ ]:
llm = ChatOpenAI(model_name='gpt-3.5-turbo')
chain = load_qa_chain(llm, chain_type='stuff')

In [ ]:
pregunta = pregunta
respuesta = chain.run(input_documents=docs, question=pregunta)

NameError: ignored

In [ ]:
print(wrap_text_preserve_newlines(respuesta))

En la nomenclatura IOB (Inside-Outside-Beginning), un chunk es una secuencia de tokens que conforman una
entidad. En otras palabras, un chunk es un trozo o grupo de palabras que juntas representan una entidad en un
texto.
